# COVID-19 relative to country size: world maps

I am living in the Netherlands, which is a small country. So far, I have only seen overall numbers by country, but I want to know how bad the situation is relative to the country size. Therefore, I decided to do a quick notebook.

**Although my initial intention was to do a short kernel only, I am now planning to add a lot more. Please stay tuned!**

# Table of contents
* [Adding country population to the COVID-19 figures](#Adding-country-population-to-the-COVID-19-figures)
* ["Top" 20 countries with relatively most confirmed cases](#"Top"-20-countries-with-relatively-most-confirmed-cases)
* [World map with Cases per Million for each country](#World-map-with-Cases-per-Million-for-each-country)
* ["Top" 20 countries with relatively most deaths](#"Top"-20-countries-with-relatively-most-deaths)
* [World map with Deaths per Million for each country](#World-map-with-Deaths-per-Million-for-each-country)
* [World map: Bubble chart showing info by Province/state](#World-map:-Bubble-chart-showing-info-by-Province/state)

In [ ]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', None)

import plotly.offline as py
from plotly import tools
py.init_notebook_mode(connected=True)
import plotly.express as px
import plotly.graph_objs as go

import folium

In [ ]:
#df = pd.read_csv("../input/novel-corona-virus-2019-dataset/COVID19_line_list_data.csv")
#COVID19_open_line_list = pd.read_csv("../input/novel-corona-virus-2019-dataset/COVID19_open_line_list.csv")
df = pd.read_csv("../input/novel-corona-virus-2019-dataset/covid_19_data.csv", parse_dates=['Last Update'])
df.rename(columns={'Country/Region':'Country'}, inplace=True)
df = df.drop(columns = ['SNo', "Last Update"]) #only confuses

df_conf = pd.read_csv("../input/novel-corona-virus-2019-dataset/time_series_covid_19_confirmed.csv")
df_conf.rename(columns={'Country/Region':'Country'}, inplace=True)

# time_series_covid_19_deaths = pd.read_csv("../input/novel-corona-virus-2019-dataset/time_series_covid_19_deaths.csv")
# time_series_covid_19_recovered = pd.read_csv("../input/novel-corona-virus-2019-dataset/time_series_covid_19_recovered.csv")
countries = pd.read_csv("../input/countries-of-the-world-iso-codes-and-population/countries_by_population_2019.csv")
countries_iso = pd.read_csv("../input/countries-of-the-world-iso-codes-and-population/country_codes_2020.csv")

# Adding country population to the COVID-19 figures

File "covid_19_data.csv" from dataset "Novel Corona Virus 2019 Dataset" contains info by day. Below, I am checking what the info looks like for my own country.

In [ ]:
df[df.Country == "Netherlands"].sort_values(['ObservationDate'], ascending = False)

This seems correct to me! If I just keep the most recent line, I do get the most recent, cumulative numbers per State/Country indeed. All I have to do then is to consolidate those per Country (for instance add up all numbers of the US States). Below you can see a sample of the resulting dataframe.

In [ ]:
#strip white spaces are there is one country (Azerbaijan) with a whitespace observation
df['Country'] = df['Country'].str.strip()

#fill missing Province/State with Country
df.loc[df['Province/State'].isnull(), 'Province/State'] = df.loc[df['Province/State'].isnull(), 'Country']

#keep most recent line per Province/State and Country
df.sort_values(['Country', 'Province/State', 'ObservationDate'], ascending = [True,True,False], inplace = True)
df = df.drop_duplicates(['Country', 'Province/State'], keep = "first")

#keep a copy for later on
df_state = df.copy()

df = df.drop(columns = "ObservationDate")

#groupby Country
df_country = df.groupby(['Country'], as_index=False)['Confirmed', 'Deaths'].sum()

df_country.sample(5)

Dataset "Countries of the World; ISO codes and population", uploaded by me, contains the number of inhabitants by country. As I want to use built-in geometries of plotly.express later on to plot numbers on a world map, I am also merging the three-letter ISO country code to the dataframe.

In [ ]:
#drop some columns
cols_to_drop = ['Rank', 'pop2018','GrowthRate', 'area', 'Density']
countries = countries.drop(columns = cols_to_drop)

#add ISO Alpha 3 code that I uploaded in another CSV
countries = countries.merge(countries_iso[['name', 'cca3']], on = ['name'], how = "left")

cols_to_rename = {'name': 'Country', 'pop2019': 'Population', 'cca3': 'ISO'}
countries = countries.rename(columns = cols_to_rename)

countries.head()

This enables me to add the population numbers and ISO codes to the dataframe with Corona figures by country, and calculate the 'Cases per million inhabitants' and 'Deaths per million inhabitants'. Below you can see a sample of the resulting dataframe.

In [ ]:
#just fixing the most important mismatches
countries_to_rename = {'US': 'United States',\
                       'Mainland China': 'China',\
                       'UK': 'United Kingdom',\
                       'Congo (Kinshasa)': 'DR Congo',\
                       'North Macedonia': 'Macedonia',\
                       'Republic of Ireland': 'Ireland',\
                       'Congo (Brazzaville)': 'Republic of the Congo'}

df_country['Country'] = df_country['Country'].replace(countries_to_rename)

df_country = df_country.merge(countries[['Country', 'Population', 'ISO']], on = "Country", how = "left")

#check mismatches
#df_country[df_country.ISO.isnull()].sort_values(['Confirmed'], ascending = False)

#dropping not matching countries, only small islands left
df_country = df_country.dropna()

#rounding population to millions with 2 digits, and creating two new columns
df_country['Population'] = round((df_country['Population']/1000),2)
df_country = df_country.rename(columns = {'Population': 'Population (million)'})
df_country['Cases per Million'] = round((df_country['Confirmed']/df_country['Population (million)']),2)
df_country['Deaths per Million'] = round((df_country['Deaths']/df_country['Population (million)']),2)

#filter out countries with less than a million population as for instance San Marino has extremely high figures on a very small population
df_country = df_country[(df_country['Population (million)'] > 1)]

df_country.sample(5)

# "Top" 20 countries with relatively most confirmed cases

Final relative ranking sorted on 'Cases per Million' (20 countries with most cases per million only). In this list, countries with less than a million inhabitants are excluded.

In [ ]:
df_country = df_country.sort_values(['Cases per Million'], ascending = False).reset_index(drop=True)
df_country.drop(columns = ['ISO', 'Deaths', 'Deaths per Million']).head(20)

# World map with Cases per Million for each country

*Hovering over the map below shows the info in a tooltip*

In [ ]:
fig = px.choropleth(df_country, locations="ISO",
                    color="Cases per Million",
                    hover_name="Country",
                    color_continuous_scale=px.colors.sequential.YlOrRd)

layout = go.Layout(
    title=go.layout.Title(
        text="Corona confirmed cases per million inhabitants",
        x=0.5
    ),
    font=dict(size=14),
    width = 750,
    height = 350,
    margin=dict(l=0,r=0,b=0,t=30)
)

fig.update_layout(layout)

fig.show()

# "Top" 20 countries with relatively most deaths

Final relative ranking sorted on 'Deaths per Million'. This paints a very different picture. Countries like Denmark have very few death with a lot of confirmed cases (likely: more testing of people feeling sick). Italy is even more "ahead", with a brutal mortality rate when compared to all other countries.

Again, countries with less than a million inhabitants are excluded in this list.

In [ ]:
df_country = df_country.sort_values(['Deaths per Million'], ascending = False).reset_index(drop=True)
df_country.drop(columns = ['ISO', 'Confirmed', 'Cases per Million']).head(20)

# World map with Deaths per Million for each country

*Hovering over the map below shows the info in a tooltip*

In [ ]:
fig = px.choropleth(df_country, locations="ISO",
                    color="Deaths per Million",
                    hover_name="Country",
                    color_continuous_scale=px.colors.sequential.YlOrRd)

layout = go.Layout(
    title=go.layout.Title(
        text="Corona deaths per million inhabitants",
        x=0.5
    ),
    font=dict(size=14),
    width = 750,
    height = 350,
    margin=dict(l=0,r=0,b=0,t=30)
)

fig.update_layout(layout)

fig.show()

# World map: Bubble chart showing info by Province/state

The CSV with the time series of confirmed cases contains coordinates that I can use to plot on a map, but I first want to check if the file is as up-to-date as the dataframe that I have used so far. To do so, I am only displaying the last 5 columns added and filtering on the Netherlands.

In [ ]:
#get names of first 4 and last 5 columns
cols_to_select = list(df_conf.columns[0:4]) + list(df_conf.columns[(len(df_conf.columns)-6): (len(df_conf.columns)-1)])

df_conf.loc[(df_conf['Country'] == "Netherlands"), cols_to_select]

As you can see below, the dataframe that I have been working with so far is more up-to-date

In [ ]:
df_state[(df_state['Country'] == "Netherlands")]

What all this allows me to do is add the coordinates from the time series dataframe to my dataframe.

In [ ]:
#also replace NaN in df_conf
#fill missing Province/State with Country
df_conf.loc[df_conf['Province/State'].isnull(), 'Province/State'] = df_conf.loc[df_conf['Province/State'].isnull(), 'Country']

cols_to_merge = ['Province/State', 'Country', 'Lat', 'Long']

df_state = df_state.merge(df_conf[cols_to_merge], on = ['Province/State', 'Country'], how = "inner")
#df_state[['Confirmed', 'Deaths', 'Recovered']] = df_state[['Confirmed', 'Deaths', 'Recovered']].astype('int')
df_state.head()

Below you can see the resulting bubble chart. Not the prettiest map that I have ever made, but the the labels that show up when hovering over the map do show details such as the info for specific States or County's in the US.

In [ ]:
m = folium.Map(location=[45, 5], zoom_start=3)

for i in range(0,len(df_state)):
   folium.Circle(
      location=[df_state.iloc[i]['Lat'], df_state.iloc[i]['Long']],
      tooltip= 'Region: {}, Confirmed Cases: {}'.format(df_state.iloc[i]['Province/State'], df_state.iloc[i]['Confirmed'].astype(int).astype(str)),
      radius=df_state.iloc[i]['Confirmed']*20,
      color='crimson',
      fill=True,
      fill_color='crimson'
   ).add_to(m)

m

To be continued. Please stay tuned!